In [1]:
import sys
sys.path.append('../')

import feature_utils as feat_util
import fetch_data as data_getter
import clean_data as cleaner
import pandas as pd

## Sample testing

In [2]:
pdd = data_getter.get_all_player_data()
fixture_data = data_getter.get_all_fixture_df()

TypeError: get_all_player_data() missing 1 required positional argument: 'fixtures'

In [ ]:
from datetime import datetime
import pytz

fixture_data['kickoff_time'] = pd.to_datetime(fixture_data['kickoff_time'])
current_time = datetime.utcnow()
current_time = current_time.replace(tzinfo=pytz.UTC)

nxt_fxtr = fixture_data[fixture_data['kickoff_time'] > current_time]

current_gameweek = nxt_fxtr['event'].min()
current_gameweek.min()

10.0

In [53]:
# Fixing get future gameweeks function
num_gameweeks = 2

current_time = datetime.utcnow()
current_time = current_time.replace(tzinfo=pytz.UTC)
futr_fxtr = fixture_data[fixture_data['kickoff_time'] > current_time]

next_gameweeks = sorted(futr_fxtr['event'].unique())[:num_gameweeks]

nxt_gw_fxtr = futr_fxtr[futr_fxtr['event'].isin(next_gameweeks)]

players_played_last_gw = pdd[(pdd['gameweek'] == nxt_gw_fxtr['event'].unique()[0] - 2) & (pdd['season'] == '2023-24')]

fut_gw_players = []
for gw in next_gameweeks:
    gw_x_players = players_played_last_gw.copy()
    gw_x_players['gameweek'] = gw
    fut_gw_players.append(gw_x_players)

players_played_last_gw = pd.concat(fut_gw_players)

In [54]:
# Update the rest of the information based on the next fixture for the team
import numpy as np

drop_cols = ['difficulty', 'opponent_difficulty', 'opponent_team_name', 'opponent_team', 'kickoff_time']
players_played_last_gw.drop(columns=drop_cols, inplace=True)
players_played_last_gw['minutes'] = players_played_last_gw['minutes'].apply(lambda x: 60 if x <= 60 else 90)
next_gw = players_played_last_gw.merge(nxt_gw_fxtr[['event', 'team_name', 'id', 'kickoff_time', 'opponent', 'difficulty', 'opponent_difficulty', 'opponent_name']],
                                            left_on=['team', 'gameweek'],
                                            right_on=['team_name', 'event'],
                                            how='left')
next_gw['kickoff_time'] = pd.to_datetime(next_gw['kickoff_time'])
next_gw['gameweek'] = next_gw['event']
next_gw['opponent_team'] = next_gw['opponent']
next_gw['opponent_team_name'] = next_gw['opponent_name']
next_gw['fixture'] = next_gw['id']
next_gw.drop(columns=['event', 'id', 'opponent', 'opponent_name', 'team_name'], inplace=True)
cols_to_nan = ['goals_scored', 'assists', 'clean_sheets', 'goals_conceded', 'penalties_missed',
            'penalties_saved', 'yellow_cards', 'red_cards', 'saves', 'bonus', 'bps', 
            'total_points', 'transfers_balance', 'transfers_in', 'transfers_out',
            'selected', 'own_goals', 'score', 'opponent_score']
next_gw[cols_to_nan] = np.nan

In [58]:
next_gw

,name,position,team,assists,bonus,bps,clean_sheets,creativity,element,fixture,...,season,gameweek,starts,score,opponent_score,kickoff_time,difficulty,opponent_difficulty,opponent_team,opponent_team_name
0,Femi Seriki,DEF,Sheffield Utd,NaN,NaN,NaN,NaN,0.0,653,91,...,2023-24,10.0,0.0,NaN,NaN,2023-10-28 14:00:00+00:00,4,2,1,Arsenal
1,Josh Brooking,DEF,Chelsea,NaN,NaN,NaN,NaN,0.0,723,95,...,2023-24,10.0,0.0,NaN,NaN,2023-10-28 11:30:00+00:00,3,3,4,Brentford
2,Radek Vítek,GK,Man Utd,NaN,NaN,NaN,NaN,0.0,669,98,...,2023-24,10.0,0.0,NaN,NaN,2023-10-29 15:30:00+00:00,5,4,13,Man City
3,Jack Hinshelwood,MID,Brighton,NaN,NaN,NaN,NaN,0.0,621,94,...,2023-24,10.0,0.0,NaN,NaN,2023-10-29 14:00:00+00:00,2,3,10,Fulham
4,Jadon Sancho,MID,Man Utd,NaN,NaN,NaN,NaN,0.0,397,98,...,2023-24,10.0,0.0,NaN,NaN,2023-10-29 15:30:00+00:00,5,4,13,Man City
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1445,Kyle Walker,DEF,Man City,NaN,NaN,NaN,NaN,6.1,369,106,...,2023-24,11.0,1.0,NaN,NaN,2023-11-04 15:00:00+00:00,2,5,3,Bournemouth
1446,Jacob Brown,FWD,Luton,NaN,NaN,NaN,NaN,0.8,631,105,...,2023-24,11.0,1.0,NaN,NaN,2023-11-05 16:30:00+00:00,4,2,11,Liverpool
1447,Vicente Guaita,GK,Crystal Palace,NaN,NaN,NaN,NaN,0.0,227,102,...,2023-24,11.0,0.0,NaN,NaN,2023-11-04 15:00:00+00:00,2,2,6,Burnley
1448,Braian Ojeda Rodríguez,MID,Nott'm Forest,NaN,NaN,NaN,NaN,0.0,459,108,...,2023-24,11.0,0.0,NaN,NaN,2023-11-05 14:00:00+00:00,3,2,2,Aston Villa


## Model test

In [2]:
import sys
sys.path.append('../')

from rf_model import rf_xp_model
import feature_utils as feat_util
import fetch_data as data_getter
import clean_data as cleaner
import pandas as pd

In [3]:
# SLOW ASS FUNCTIONS
fix = data_getter.get_all_fixture_df()
pdd = data_getter.get_all_player_data(fix)
future = data_getter.get_future_gameweeks(pdd, fix)
all_df = pd.concat([pdd, future])

pdd.shape, future.shape, all_df.shape, all_df.columns

Gameweek 12 for season 2023-24 hasn't been played yet.
14.0


((106676, 42),
 (733, 42),
 (107409, 42),
 Index(['name', 'position', 'team', 'assists', 'bonus', 'bps', 'clean_sheets',
        'creativity', 'element', 'fixture', 'goals_conceded', 'goals_scored',
        'ict_index', 'influence', 'minutes', 'opponent_team', 'own_goals',
        'penalties_missed', 'penalties_saved', 'red_cards', 'round', 'saves',
        'selected', 'team_a_score', 'team_h_score', 'threat', 'total_points',
        'transfers_balance', 'transfers_in', 'transfers_out', 'value',
        'was_home', 'yellow_cards', 'season', 'gameweek', 'starts',
        'kickoff_time', 'difficulty', 'opponent_difficulty', 'score',
        'opponent_score', 'opponent_team_name'],
       dtype='object'))

In [4]:
player_df= cleaner.clean_player_data(all_df, data_getter.get_all_fixture_df())
player_df = feat_util.generate_difficulty_feature(player_df)
player_df = feat_util.encode_positions(player_df)
player_df = feat_util.generate_rolling_form(player_df)
player_df = feat_util.get_rolling_avg_mins(player_df)
player_df = feat_util.generate_rolling_ict(player_df)
player_df = feat_util.get_avg_ppg(player_df)
player_df = feat_util.encode_teams(player_df)

cols_to_remove = ['assists', 'bonus', 'bps', 'clean_sheets', 'creativity', 
                  'goals_conceded', 'goals_scored', 'ict_index', 'influence', 'own_goals',
                  'penalties_missed', 'penalties_saved', 'red_cards', 'round', 'saves', 
                  'team_a_score', 'team_h_score', 'threat', 'transfers_balance', 
                  'transfers_in', 'transfers_out','yellow_cards', 'starts', 'score', 
                  'opponent_score', 'difficulty_category']
player_df = cleaner.drop_cols(cols_to_remove, player_df)

/home/dheyaydesai/Documents/projs/FPL/weighted_through_ball/notebooks/../feature_utils.py:45: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df[f'avg_points_last_{window}'] = df.groupby('name')['total_points'].apply(lambda x: x.shift(1).rolling(window=window, min_periods=1).mean())
/home/dheyaydesai/Documents/projs/FPL/weighted_through_ball/notebooks/../feature_utils.py:45: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .group

In [5]:
player_df.columns

Index(['name', 'element', 'fixture', 'minutes', 'opponent_team', 'selected',
       'total_points', 'value', 'was_home', 'season', 'gameweek',
       'kickoff_time', 'difficulty', 'opponent_difficulty',
       'avg_points_against_difficulty', 'position_DEF', 'position_FWD',
       'position_GK', 'position_MID', 'avg_points_last_3', 'avg_points_last_5',
       'avg_points_last_10', 'avg_minutes_last_1', 'avg_minutes_last_2',
       'avg_minutes_last_3', 'avg_minutes_last_4', 'avg_minutes_last_5',
       'influence_avg_last_3_games', 'creativity_avg_last_3_games',
       'threat_avg_last_3_games', 'avg_points_upto_gw', 'team_Arsenal',
       'team_Aston_Villa', 'team_Bournemouth', 'team_Brentford',
       'team_Brighton', 'team_Burnley', 'team_Chelsea', 'team_Crystal_Palace',
       'team_Everton', 'team_Fulham', 'team_Leeds', 'team_Leicester',
       'team_Liverpool', 'team_Luton', 'team_Man_City', 'team_Man_Utd',
       'team_Newcastle', 'team_Norwich', 'team_Nott'm_Forest',
       'te

In [6]:
samp_df = player_df.copy()
samp_df = pd.get_dummies(samp_df, columns=['season'])

len(player_df['name'].unique()), len(player_df['element'].unique())

(1495, 1495)

In [7]:
# Filter data to keep only rows where the kickoff time is in the past
integer_rows = samp_df[samp_df['kickoff_time'].apply(lambda x: isinstance(x, int))]
cleaned_df = future.copy()
nan_counts = cleaned_df.isna().sum()
nan_counts = nan_counts[nan_counts > 0]
print(nan_counts)

assists              733
bonus                733
bps                  733
clean_sheets         733
goals_conceded       733
goals_scored         733
own_goals            733
penalties_missed     733
penalties_saved      733
red_cards            733
saves                733
selected             733
total_points         733
transfers_balance    733
transfers_in         733
transfers_out        733
yellow_cards         733
score                733
opponent_score       733
dtype: int64


In [8]:
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import matplotlib.pyplot as plt

# Filter data to keep only rows where the kickoff time is in the past
current_time = datetime.utcnow()
import pytz
current_time = current_time.replace(tzinfo=pytz.UTC)

train_data = samp_df[samp_df['kickoff_time'] <= current_time]
test_data = samp_df[samp_df['kickoff_time'] > current_time]

X_test = test_data.drop(columns=['total_points'])
y_test = test_data['total_points']

X_train = train_data.drop(columns=['total_points'])
y_train = train_data['total_points']


train_data.shape, X_test.shape

((84116, 86), (733, 85))

In [9]:
model = rf_xp_model()

model.train_model(X_train, y_train)

/home/dheyaydesai/anaconda3/envs/272class/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


Mean Squared Error: 2.937721437210399
Mean Absolute Error: 0.7659945688824927


In [10]:
xP = model.predict(X_test.drop(columns=['name', 'kickoff_time']))

xP

array([ 2.62139011e+00,  4.54875118e+00,  3.29442762e+00,  5.22155899e+00,
        2.28233116e+00,  2.36920132e+00,  4.15272594e+00,  7.20754853e-02,
        2.32444037e+00,  2.71885404e+00,  3.58549979e+00,  3.90208343e+00,
        2.45979023e+00,  6.14602924e-01,  4.90206340e-01,  4.61124420e+00,
       -4.61433287e-01,  2.37571551e+00,  2.24317410e+00,  8.05782465e-01,
        3.33397539e+00,  2.93766302e+00,  8.22818931e-01,  2.43261002e+00,
        3.89402381e-01,  6.06316173e-01, -1.80293567e-01,  6.08099046e+00,
        4.54036159e+00,  3.76399272e+00,  1.44722368e+00,  5.18421519e-02,
        1.24971997e+00,  5.36535347e+00,  5.70038224e-01,  4.34490335e+00,
        3.21033613e+00,  8.83553922e-01,  2.74899903e+00,  3.33832464e+00,
        1.22525554e+00,  2.89861147e+00,  2.80979742e+00,  2.42170612e+00,
        2.68756167e+00,  4.39583483e+00,  2.72311325e+00,  4.27825391e+00,
        2.55035219e+00,  2.63767040e+00,  2.14918661e+00,  2.37464015e+00,
        2.40122535e+00,  

In [11]:
from team_opt import *

test_data = X_test.copy()
test_data['xP'] = xP

data = test_data.copy()

In [12]:
test_data[test_data['team_Man_City'] == 1][['name', 'xP']]

,name,xP
7604,Aymeric Laporte,4.364603
9725,Bernardo Mota Veiga de Carvalho e Silva,6.043452
21950,Ederson Santana de Moraes,4.747030
24142,Erling Haaland,7.195641
32725,Jack Grealish,4.110485
40492,John Stones,4.126687
43162,João Cancelo,4.114802
43730,Joško Gvardiol,3.563070
44022,Julián Álvarez,5.128720
44170,Jérémy Doku,6.960256


In [23]:
for i, j in test_data.iterrows():
    print(i, j['element'], j['name'])

87 253 Aaron Connolly
213 418 Aaron Cresswell
263 512 Aaron Hickey
462 221 Aaron Ramsdale
511 684 Aaron Ramsey
637 660 Aaron Wan-Bissaka
763 312 Abdoulaye Doucouré
775 123 Abdul Rahman Baba
910 609 Adam Davies
1188 359 Adam Lallana
1276 654 Adam Smith
1402 620 Adam Webster
1489 239 Adama Traoré
1662 241 Ademola Ola-Adebomi
1674 333 Admiral Muskwe
1880 458 Adrián San Miguel del Castillo
1968 338 Aidan Francis-Clarke
2193 120 Albert Sambi Lokonga
2281 320 Alejandro Garnacho
2300 345 Alejo Véliz
2388 215 Aleksandar Mitrović
2514 355 Alex Iwobi
2562 669 Alex Matos
2726 243 Alex Mighten
2738 31 Alex Murphy
2940 705 Alex Scott
3026 637 Alex Telles
3073 474 Alexander Isak
3201 282 Alexandre Moreno Lopera
3327 246 Alexis Mac Allister
3410 571 Alfie Doughty
3421 268 Alfie Gilchrist
3471 100 Alfie Whiteman
3673 362 Alisson Ramses Becker
3685 269 Allan Campbell
3925 395 Allan Saint-Maximin
4072 361 Alphonse Areola
4081 618 Altay Bayindir
4175 22 Amad Diallo
4224 457 Amadou Onana
4236 467 Amari'i 

In [25]:
import pandas as pd
import pulp

# Assuming df is your DataFrame and it includes the 'xP' column for expected points
df = test_data.copy()

# Define the problem
prob = pulp.LpProblem('FantasyFootballTeam', pulp.LpMaximize)

# Decision variables
player_vars = {row['element']: pulp.LpVariable(f"player_{row['element']}", cat='Binary') for i, row in df.iterrows()}


# Objective function: Maximize the sum of expected points
prob += pulp.lpSum([player_vars[row['element']] * row['xP'] for i, row in df.iterrows()])

# Constraints

# Team size constraints
prob += pulp.lpSum(player_vars.values()) == 15, "Total_15_Players"
prob += pulp.lpSum(player_vars[row['element']] * row['position_GK'] for i, row in df.iterrows()) == 2, "2_Goalkeepers"
prob += pulp.lpSum(player_vars[row['element']] * row['position_DEF'] for i, row in df.iterrows()) >= 3, "Atleast_3_Defenders"
prob += pulp.lpSum(player_vars[row['element']] * row['position_DEF'] for i, row in df.iterrows()) <= 5, "Max_5_Defenders"
prob += pulp.lpSum(player_vars[row['element']] * row['position_MID'] for i, row in df.iterrows()) >= 3, "Atleast_3_Midfielders"
prob += pulp.lpSum(player_vars[row['element']] * row['position_MID'] for i, row in df.iterrows()) <= 5, "Max_5_Midfielders"
prob += pulp.lpSum(player_vars[row['element']] * row['position_FWD'] for i, row in df.iterrows()) >= 1, "Atleast_1_Forward"
prob += pulp.lpSum(player_vars[row['element']] * row['position_FWD'] for i, row in df.iterrows()) <= 3, "Max_3_Forwards"

# Budget constraint
prob += pulp.lpSum(player_vars[row['element']] * row['value'] for i, row in df.iterrows()) <= 1000, "Total_Budget"

# Team diversity constraints (max 3 players from the same team)
# Extract the team names from the columns
team_columns = [col for col in df.columns if col.startswith('team_')]
teams = [col.replace('team_', '') for col in team_columns]
for team in teams:  # teams should be a list of all team names
    prob += pulp.lpSum(player_vars[row['element']] * row[f'team_{team}'] for i, row in df.iterrows()) <= 3, f"Max_3_from_{team}"

# Solve the problem
prob.solve()

# Output the selected players
selected_players = {i for i in player_vars if player_vars[i].value() == 1}
selected_df = df[df['element'].isin(selected_players)]


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/dheyaydesai/anaconda3/envs/272class/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/477aa0228b0b48669a736af7bb72d4e6-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/477aa0228b0b48669a736af7bb72d4e6-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 40 COLUMNS
At line 5814 RHS
At line 5850 BOUNDS
At line 6583 ENDATA
Problem MODEL has 35 rows, 732 columns and 3577 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 93.697 - 0.00 seconds
Cgl0004I processed model has 26 rows, 728 columns (728 integer (724 of which binary)) and 2913 elements
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.384615
Cbc0038I Pass   1: suminf.    0.10526 (2) obj. -93.4388 iterations 11
Cbc0038I Pass   2: suminf.    0.00000 (0) obj. -87.0863 iterations 1
Cbc0038I Solutio

In [27]:
print(f"Total expected points: {pulp.value(prob.objective)}")
print(f"Selected players:")
selected_df[['name', 'element', 'position_GK', 'position_DEF', 'position_MID', 'position_FWD', 'value', 'xP']]

Total expected points: 93.57776103697667
Selected players:


,name,element,position_GK,position_DEF,position_MID,position_FWD,value,xP
3073,Alexander Isak,474,0,0,0,1,74,6.080990
16460,Cédric Alves Soares,477,0,1,0,0,39,6.063592
24142,Erling Haaland,496,0,0,0,1,140,7.195641
27436,Gabriel Martinelli Silva,230,0,0,1,0,77,7.157278
44170,Jérémy Doku,549,0,0,1,0,65,6.960256
45090,Karl Hein,252,1,0,0,0,40,6.051098
46188,Kevin De Bruyne,504,0,0,1,0,102,7.101026
46850,Kieran Trippier,296,0,1,0,0,70,6.076671
51048,Lukasz Fabianski,291,1,0,0,0,42,5.222203
59961,Moussa Diaby,570,0,0,1,0,68,6.410446


In [28]:
test_data[test_data['name'] == 'Karl Hein']

,name,element,fixture,minutes,opponent_team,selected,value,was_home,gameweek,kickoff_time,...,opponent_team_Spurs,opponent_team_Watford,opponent_team_West_Brom,opponent_team_West_Ham,opponent_team_Wolves,season_2020-21,season_2021-22,season_2022-23,season_2023-24,xP
45090,Karl Hein,252,131,60,20,0,40,False,14.0,2023-12-02 15:00:00+00:00,...,0,0,0,0,1,0,0,0,1,6.051098


In [ ]:
## TODO -- Fix optimizer

## TODO -- Fix playe availability predictor
# Get upto date information on player availablility at time of prediction for more upto date and better predictions, so far so good